In [1]:
import cv2
import numpy as np

from tracker import DAT_TRACKER
import os

In [2]:

class DatCfg:
    def __init__(self):
        self.show_figures = False
        self.img_scale_target_diagonal = 75
        self.search_win_padding = 2.0
        self.surr_win_factor = 1.9
        self.color_space = 1  # 1: RGB, 2: LAB, 3: HSV, 4: GRAY
        self.num_bins = 16
        self.bin_mapping = self.get_bin_mapping(self.num_bins)  # Initialize bin mapping
        self.prob_lut_update_rate = 0.05
        self.distractor_aware = True
        self.adapt_thresh_prob_bins = np.linspace(0, 1, num=21)  # 0:0.05 : 1
        self.motion_estimation_history_size = 5
        self.nms_scale = 1
        self.nms_overlap = 0.9
        self.nms_score_factor = 0.5
        self.nms_include_center_vote = True

    def get_bin_mapping(self, num_bins):
        # Replace this with the actual bin mapping logic
        # For example, it could create a 1D array of size num_bins
        return np.array([255 - i for i in range(256)], dtype=np.uint8)

# Example usage
cfg = DatCfg()
cfg_dict = vars(cfg)

In [3]:
cfg_dict = vars(cfg)
print("Color Space (Dictionary Indexing):", cfg_dict['color_space'])

Color Space (Dictionary Indexing): 1


In [4]:


import glob
def poly2rect(polygon):
    """Convert a polygon (4 points) to a bounding rectangle."""
    # Get the bounding box of the polygon
    x, y, w, h = cv2.boundingRect(np.array(polygon, dtype=np.float32))
    return (x, y, w, h)

def get_ground_truth(txt_file):
    rects = []
    
    try:
        with open(txt_file, 'r') as gt:
            for line in gt:
                # Replace commas with spaces and split the line into components
                line = line.replace(',', ' ').strip()
                coords = list(map(float, line.split()))
                
                # Check if there are exactly 8 coordinates (4 points)
                if len(coords) == 8:
                    polygon = [
                        (coords[0], coords[1]),
                        (coords[2], coords[3]),
                        (coords[4], coords[5]),
                        (coords[6], coords[7])
                    ]
                    rect = poly2rect(polygon)  # Get the bounding rect
                    rects.append(rect)  # Append to the list
                else:
                    print(f"Line does not contain 8 coordinates: {line}")

    except IOError:
        print(f"Ground truth file {txt_file} cannot be read")

    return rects

def main():
    video_base_path = ""
    pattern_jpg = "sequence"
    txt_base_path = "sequence/groundtruth.txt"
    images = glob.glob(os.path.join(pattern_jpg, '*.jpg'))
    image_files = sorted(images)
    if len(image_files) == 0:
        print("No image files found")
        return
    
    groundtruth_rect = get_ground_truth(txt_base_path)
    
    dat = DAT_TRACKER(cfg_dict)
    location = groundtruth_rect[0]
    result_rects = []
    time_total = 0
    
    for frame, image_file in enumerate(image_files):
        image = cv2.imread(image_file)
        tic = cv2.getTickCount()
        
        if frame == 0:
            dat.tracker_dat_initialize(image, location)
        else:
            location = dat.tracker_dat_update(image)
        
        toc = cv2.getTickCount() - tic
        time_total += toc
        result_rects.append(location)
        
        # Visualization
        cv2.putText(image, str(frame + 1), (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        cv2.rectangle(image, tuple(groundtruth_rect[frame]), (0, 255, 0), 2)
        cv2.rectangle(image, tuple(location), (0, 128, 255), 2)
        cv2.imshow("DAT", image)

        if cv2.waitKey(1) & 0xFF in [27, ord('q'), ord('Q')]:
            break
    
    fps = len(result_rects) / (time_total / cv2.getTickFrequency())
    print(f"FPS: {fps:.2f}")
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


(16,)


ValueError: bin_mapping must be a 1D array with 256 elements